Notes from [Introduction to Synthetic Aperture Radar Using Python and MATLAB](https://ieeexplore.ieee.org/document/9893146).

# BackProjection

Compared to the range-Doppler and polar format alogirthms, backprojection does not need to make as many assumptions, such as about cell migration or platform motion. Its implementation is basically the same regardless of the imaging method. Backprojection also forms images in the ground plane, which makes orthorectification and registration easier and more efficient.

### The Fourier Slice Theorem and Radon Transforms

<img src="imgs/fourier_slice_theorem.png" width="700" height="700" style='margin-left: auto; margin-right: auto;' />

In the above image, showing the geometry of X-ray tomography, the X-ray paths form parallel projections of the density function $f(x, y)P_{\theta}(t)$. According to the Fourier Slice Theorem, the 1-D Fourier Transform of of a parallel projection of a density function is a slice of the 2-D Fourier Transform of the density function. Therefore, if one gets these projections from multiple angles, they get get back to the original image.
<br>
<br>
The time domain signal generated by the X-rays is the 1-D projection of the density function, and is defined by the Radon Transform. The Radon Transfrom is given by
$$ P_{\theta}(t) = \int_{l} f(x, y)ds = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty}f(x, y)\delta(x\cos(\theta)+y\sin(\theta)-t)dxdy) $$
The 1-D Fourier Transform of this gives us
$$ S_{\theta}(\omega) = \int_{-\infty}^{\infty}P_{\theta}e^{-i2\pi \omega t}dt $$
and the 2-D Fourier Transform of the orginal density function is
$$ F(u, v) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty}f(x, y)e^{-i2\pi (ux + vy)}dxdy $$
Since no assumptions about the orientation are needed, $ \theta $ and $ v $ can be 0. This gives us
$$ F(u, 0) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty}f(x, y)e^{-i2\pi ux}dxdy $$
Since the phase term is no only a function of x, the integral can now become
$$ F(u, 0) = \int_{-\infty}^{\infty}[\int_{-\infty}^{\infty}f(x, y)]e^{-i2\pi ux}dx $$
The term inside the bracks is then a projection across lines of constant $x$, so
$$ P_{\theta = 0} = \int_{-\infty}^{\infty}f(x, y)dy $$
and
$$ F(u, 0) = \int_{-\infty}^{\infty}P_{\theta = 0}e^{-i2\pi ux}dx $$
This shows that a slice of the 2-Fourier transform at any particular angle is the 1-D Fourier transform of the density function in that direction. Since we made no assumption about the particular angle, we can see that across all angles we end up with
$$ F(u, 0) = S_{\theta = 0}(\omega) $$
$$ F(u, v) = S_{\theta}(\omega) $$

### Filtered Backprojection

Once the 1-D Fourier Transforms of the projections have been collected, we have $F(u, v)$. So we can take the 2-D inverse Fourier transform of this frequency domain representation of the density function to get the density function. This can be written as 
$$ f(x, y) =  \int_{-\infty}^{\infty}\int_{-\infty}^{\infty}F(u, v)e^{i2\pi[ux + vy]}dudv $$
Since the projection functions cover the spatial frequency domain in a polar fashion, the following substatutions can be made
$$ u = \omega \cos(\theta) $$
$$ v = \omega \sin(\theta) $$
$$ dudv = \omega d\omega d\theta $$
which gives us
$$ f(x, y) =  \int_{0}^{2\pi}\int_{0}^{\infty}F(\omega, \theta)e^{i2\pi \omega[x\cos(\theta) + y\sin(\theta)]}\omega d\omega d\theta $$
This can be split into two integrals like
$$ f(x, y) =  \int_{0}^{\pi}\int_{0}^{\infty}F(\omega, \theta)e^{i2\pi \omega[x\cos(\theta) + y\sin(\theta)]}\omega d\omega d\theta + \int_{0}^{\pi}\int_{0}^{\infty}F(\omega, \theta)e^{i2\pi \omega[x\cos(\theta + \pi) + y\sin(\theta + \pi)]}\omega d\omega d\theta $$
Then, using $ F(\omega, \theta + \pi) = F(-\omega, \theta) $ we can get
$$ f(x, y) =  \int_{0}^{\pi}\int_{0}^{\infty}F(\omega, \theta)e^{i2\pi \omega[x\cos(\theta) + y\sin(\theta)]}|\omega| d\omega $$
The inner integral is the convolution of the projection with a filter like
$$ P_{\theta}(t)*h(t) =  \int_{-\infty}^{\infty}F(\omega, \theta)e^{i2\pi[x\cos(\theta) + y\sin(\theta)]}|\omega| d\omega $$
This filtered projection function is the projected back onto the image space along the line the projection was obtained. This can be written as 
$$ f(x, y) = \int_{0}^{\pi}P_{\theta}(t)*h(t)d\theta $$

While all of this math refers to the geometry of X-ray tomography, it works for synthetic aperture radar as well. It works by processing the incoming signal as a projection of the reflectivity function of the target onto the signal. This projection data is the gather across multiple look angles to cover the target's spatial frequency domain. Filtered Backprojection is then used to reconstruct the image from the spatial frequency data.